In [1]:
import requests
import os
from importlib.machinery import SourceFileLoader
import pandas as pd
import logging
import torch


try:
    config = SourceFileLoader("config", "config.py").load_module()
    os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

    bungie_api_key = config.BUNGIE_API_KEY
    endpoint = "https://www.bungie.net/Platform/Content/Rss/NewsArticles/{pageToken}/"
    page_token = "0"
    include_body = True
    headers = {
        "X-API-Key": bungie_api_key
    }
    params = {
        "includebody": include_body
    }

    results = []

    while page_token is not None:
        response = requests.get(endpoint.format(pageToken=page_token), headers=headers, params=params)

        if response.status_code == 200:
            json_response = response.json()
            if 'NewsArticles' in json_response['Response']:
                results.extend(json_response['Response']['NewsArticles'])
            else:
                print("No NewsArticles found in the response.")
                break
            page_token = json_response['Response']['NextPaginationToken']
        else:
            print("Error:", response.status_code)
            print("Response:", response.text)
            break
except Exception as e:
    print("An error occurred:", str(e))


An error occurred: 'NextPaginationToken'


In [2]:
results[0]

{'Title': 'This Week in Destiny - 07/20/2023',
 'Link': '/7/en/News/Article/twid_07_20_23',
 'PubDate': '2023-07-20T18:00:00Z',
 'UniqueIdentifier': 'blt2e36c629958b2c26',
 'Description': 'Dance around the bonfire',
 'HtmlContent': '<p>This week in Destiny, Solstice is back, so it\'s time to&nbsp;go to the EAZ&nbsp;and earn some nice-looking armor pieces. Are you going for those perfect stats? Maybe getting the new Strand Rocket Launcher? You can also see how fast you can go around the whole arena defeating enemies using just the Strand Grapple. A certain new Strand Exotic Trace Rifle might help you navigate the perfect route...&nbsp;</p><p>Before jumping into this week\'s topics, let\'s check what we talked about <a href="https://www.bungie.net/7/en/News/article/7_13_23_twid" target="_blank">last week</a> in case you missed it:&nbsp;</p><ul><li>The date for Bungie\'s next Destiny 2 Showcase.&nbsp;</li><li>Improvements coming to the Crucible\'s Competitive playlist.&nbsp;</li><li>A sma

In [3]:
# Convert the results list into a DataFrame
df = pd.DataFrame(results)


In [4]:
import re
pattern = r'Update|Hotfix'
df = df[df['Title'].str.contains(pattern, regex=True, flags=re.IGNORECASE)]
df = df.iloc[0:25].reset_index()
# Print the filtered DataFrame
df['Title']

0                           Destiny 2 Update 7.1.5
1                         Destiny 2 Update 7.1.0.3
2                         Destiny 2 Update 7.1.0.2
3                         Destiny 2 Hotfix 6/02/23
4                         Destiny 2 Update 7.1.0.1
5                         Destiny 2 Hotfix 5/26/23
6      Destiny 2 Update 7.1.0 - Season of the Deep
7                         Destiny 2 Hotfix 7.0.5.3
8                         Destiny 2 Hotfix 7.0.5.2
9                         Destiny 2 Hotfix 7.0.5.1
10                        Destiny 2 Update 7.0.5.0
11                        Destiny 2 Hotfix 7.0.0.7
12                        Destiny 2 Hotfix 7.0.0.6
13                       Lightfall Crucible Update
14                        Destiny 2 Hotfix 7.0.0.5
15                        Destiny 2 Hotfix 7.0.0.3
16                        Destiny 2 Update 7.0.0.1
17                        Destiny 2 Hotfix 6.3.0.7
18    Economy Updates And More Coming In Lightfall
19                        Desti

In [5]:
df["HtmlContent"].iloc[0]

'<h2>Activities&nbsp;</h2><h4>CRUCIBLE&nbsp;</h4><ul><li>Adjusted spawn points on Burnout for Trials of Osiris Dominion.&nbsp;</li><li>Fixed some positions where players could get outside of the playable area on these maps:&nbsp;<ul><li>Radiant Cliffs&nbsp;</li><li>Altar of Flame&nbsp;</li></ul></li><li>Fixed an issue where the voiceover at the end of Mayhem matches would not play.&nbsp;</li></ul><h4>FREEROAM &nbsp;</h4><ul><li>Neomuna&nbsp;<ul><li>Fixed an issue where Terminal Overload could stop progressing at Stage 3.&nbsp;</li></ul></li></ul><h2>UI/UX</h2><ul><li>Fixed an issue where the Guardian Ranks icon decorations for higher ranks were missing.&nbsp; &nbsp;</li><li>Fixed an issue where Season Pass ranks past 100 were not being properly registered on the Journey screen.&nbsp;</li><li>Fixed an issue where the reward for rank 71 on the Season Pass was not unlocking properly. &nbsp;<ul><li>This issue was only visual, and the boost was properly applied when unlocked.&nbsp;</li></ul

In [6]:
from bs4 import BeautifulSoup


def extract_list_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    list_items = soup.find_all('li')
    result = [li.get_text(strip=True) for li in list_items]
    return result

def clean_html(text):
    cleantext = re.sub(r'([.,?!;:])((?!\s)|$)', r'\1 ', text)
    return cleantext

def parse_html_content(df):
    df['parsed_content'] = df['HtmlContent'].apply(extract_list_from_html)
    df['joined_content'] = df['parsed_content'].apply(lambda x: ' '.join(x))
    df['joined_content'] = df['joined_content'].apply(clean_html)
    return df

In [7]:
# Parse HTML content and create new column
df = parse_html_content(df)

In [8]:
df = df[['Title', 'Link', 'PubDate', 'UniqueIdentifier', 'Description',
       'joined_content']]

In [9]:
print(df.iloc[0]["joined_content"])

Adjusted spawn points on Burnout for Trials of Osiris Dominion. Fixed some positions where players could get outside of the playable area on these maps: Radiant CliffsAltar of Flame Radiant Cliffs Altar of Flame Fixed an issue where the voiceover at the end of Mayhem matches would not play. NeomunaFixed an issue where Terminal Overload could stop progressing at Stage 3. Fixed an issue where Terminal Overload could stop progressing at Stage 3. Fixed an issue where the Guardian Ranks icon decorations for higher ranks were missing. Fixed an issue where Season Pass ranks past 100 were not being properly registered on the Journey screen. Fixed an issue where the reward for rank 71 on the Season Pass was not unlocking properly. This issue was only visual, and the boost was properly applied when unlocked. This issue was only visual, and the boost was properly applied when unlocked. Fixed an issue where the Eververse featured carousel sound effect could be heard while previewing other items. F

In [10]:

import tiktoken  

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=50,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)


In [11]:
import os
from importlib.machinery import SourceFileLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
config = SourceFileLoader("config", "config.py").load_module()
os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

In [12]:
pinecone_api_key = config.pinecone_api_key

In [13]:
import pinecone
# connect to pinecone environment
pinecone.init(
    api_key = pinecone_api_key,
    environment = "us-west1-gcp-free"
)


c:\Users\jorda\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [14]:
index_name = "extractive-question-answering"

In [15]:
pinecone.list_indexes()

['extractive-question-answering']

In [25]:
pinecone.delete_index("extractive-question-answering")

In [26]:
# check if the extractive-question-answering index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric="cosine"
    )


In [31]:
# connect to extractive-question-answering index we created
index = pinecone.Index(index_name)

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    openai_api_key=os.environ['OPENAI_API_KEY']
)


In [29]:
docs = []
for index, row in df.iterrows():
    doc = {
        'content': row['joined_content'],  
            'name': row['Title'],
            'link': row['Link'],
            'pub_date': row['PubDate'],
            'unique_id': row['UniqueIdentifier'],
           'description': row['Description']
        }
    docs.append(doc)
 

In [32]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_limit = 100

texts = []
metadatas = []

for i, record in enumerate(tqdm(docs)):
    # first get metadata fields for this record
    metadata = {
            'name': record['name'],
            'source': record['link'],
            'pub_date': record['pub_date'],
            'unique_id': record['unique_id'],
           'description': record['description']
        }
    # now we create chunks from the record text
    record_texts = text_splitter.split_text(record['content'])
    # create individual metadata dicts for each chunk
    record_metadatas = [{
        "chunk": j, "text": text, **metadata
    } for j, text in enumerate(record_texts)]
    # append these to current batches
    texts.extend(record_texts)
    metadatas.extend(record_metadatas)
    # if we have reached the batch_limit we can add texts
    if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embed.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []


  0%|          | 0/25 [00:00<?, ?it/s]

In [33]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 720}},
 'total_vector_count': 720}

In [34]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)


In [36]:
query = "what were the changes to the immortal?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content="(crit damage goes from 21. 8 to 21). The Immortal Reduced base range value by 10. Bows Fixed an issue where the Arsenic Bite Bow would display an incorrect charge time on its tooltip. Updated the stats on the Tyranny of Heaven Bow to better compete with the current selection of Lightweight Bows. Miscellaneous and Visuals Fixed an issue where the glow when firing on the Nessa's Oblation Legendary Shotgun was not as bright as intended. Fixed an issue where the Nasreddin and", metadata={'chunk': 80.0, 'description': 'The one about Season of the Deep.', 'name': 'Destiny 2 Update 7.1.0 - Season of the Deep', 'pub_date': datetime.datetime(2023, 5, 23, 16, 30, tzinfo=tzutc()), 'source': '/7/en/News/Article/season-deep-update-7-1-0', 'unique_id': 'blt56002d0e3c869908'}),
 Document(page_content='benefit from the recent buff to their projectile collision radius. They are now consistent with other Heavy Grenade Launchers. Submachine Guns Aggressive Submachine Guns Reduced 

In [61]:
from langchain.chat_models import ChatOpenAI

# completion llm
llm = ChatOpenAI(
    openai_api_key=config.OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end include all relvant information. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectorstore.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True,
                                       chain_type="stuff")

In [62]:
# Run chain
result = qa_chain({"query": query})
print(len(result['source_documents']))
result['source_documents']

4


[Document(page_content="(crit damage goes from 21. 8 to 21). The Immortal Reduced base range value by 10. Bows Fixed an issue where the Arsenic Bite Bow would display an incorrect charge time on its tooltip. Updated the stats on the Tyranny of Heaven Bow to better compete with the current selection of Lightweight Bows. Miscellaneous and Visuals Fixed an issue where the glow when firing on the Nessa's Oblation Legendary Shotgun was not as bright as intended. Fixed an issue where the Nasreddin and", metadata={'chunk': 80.0, 'description': 'The one about Season of the Deep.', 'name': 'Destiny 2 Update 7.1.0 - Season of the Deep', 'pub_date': datetime.datetime(2023, 5, 23, 16, 30, tzinfo=tzutc()), 'source': '/7/en/News/Article/season-deep-update-7-1-0', 'unique_id': 'blt56002d0e3c869908'}),
 Document(page_content='benefit from the recent buff to their projectile collision radius. They are now consistent with other Heavy Grenade Launchers. Submachine Guns Aggressive Submachine Guns Reduced 

In [65]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

from langchain.chains import ConversationalRetrievalChain
retriever=vectorstore.as_retriever()
chat = ConversationalRetrievalChain.from_llm(llm,retriever=retriever,memory=memory)

result = chat({"question": "What were the changes to the immortal smg?"})
result['answer']

'The Immortal SMG had its base range value reduced by 10.'

In [66]:

result = chat({"question": "anything else?"})
result['answer']

'Yes, there was a change to the Immortal SMG. Its base range value was reduced by 10.'

In [ ]:
from langchain.llms import OpenAI
from langchain import LLMChain

davinci = OpenAI(model_name='text-davinci-003')

from langchain import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# create our examples
examples = [
    {
        "query": "What changes were made to the Dazzling Iridescence Trials emblem?",
        "answer": "The Dazzling Iridescence Trials emblem had an issue where it could be awarded from the Flawless chest without completing the necessary requirements. This issue has been fixed in the latest patch. Players who have already acquired the emblem will need to complete the requirements after this patch goes live in order to equip the emblem."
    }, 
    {
        "query": "What is the new change regarding Gyrfalcon's Hauberk's reserve overshield?",
        "answer": "Gyrfalcon's Hauberk's reserve overshield now deploys when a player uses Ensnaring Slam. This is a change introduced in the latest patch."
    },
    {
        "query": "What were the issues fixed regarding Arbor Warden?",
        "answer": "Two issues related to Arbor Warden were fixed in this patch. One issue caused the player to become frozen which would disable Arbor Warden's Exotic perk. The other issue was causing the Juggernaut aspect to not interact with your class ability energy correctly. Both of these issues have now been addressed."
    },
    {
        "query": "What changes were made for PvP in the latest patch?",
        "answer": "In the latest patch, several PvP-related issues were fixed. For example, Verity's Brow, which was not functioning in PvP, has been fixed. An issue where Athrys's Embrace's buff icon wasn't appearing in the HUD in PvP has also been fixed. The Collective Action weapon perk's damage against Guardians has been reduced to 10% from 20%. These are some of the changes made for PvP in this patch."
    },
    {
        "query": "Were there any changes related to 'Iron Banner Special/Power weapons'?",
        "answer": "Yes, an issue where some older Iron Banner Special/Power weapons had lost their scope glint has been fixed in this latest patch."
    },
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)


# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = "You are a knowledgeable game assistant trained to answer queries about the latest patch notes. The following are examples of how you should answer questions, but dont answer any but the last."
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

query = "What were the changes to the immortal"


example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=200  # increased max length
)

# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

print(dynamic_prompt_template.format(query=query))



You are a knowledgeable game assistant trained to answer queries about the latest patch notes. The following are examples of how you should answer questions, but dont answer any but the last.

User: What changes were made to the Dazzling Iridescence Trials emblem?
AI: The Dazzling Iridescence Trials emblem had an issue where it could be awarded from the Flawless chest without completing the necessary requirements. This issue has been fixed in the latest patch. Players who have already acquired the emblem will need to complete the requirements after this patch goes live in order to equip the emblem.


User: What is the new change regarding Gyrfalcon's Hauberk's reserve overshield?
AI: Gyrfalcon's Hauberk's reserve overshield now deploys when a player uses Ensnaring Slam. This is a change introduced in the latest patch.


User: What were the issues fixed regarding Arbor Warden?
AI: Two issues related to Arbor Warden were fixed in this patch. One issue caused the player to become frozen w

In [ ]:
query = "What were the changes to the immortal"
print(qa_with_sources(dynamic_prompt_template.format(query=query)))

{'question': "You are a knowledgeable game assistant trained to answer queries about the latest patch notes. The following are examples of how you should answer questions, but dont answer any but the last.\n\nUser: What changes were made to the Dazzling Iridescence Trials emblem?\nAI: The Dazzling Iridescence Trials emblem had an issue where it could be awarded from the Flawless chest without completing the necessary requirements. This issue has been fixed in the latest patch. Players who have already acquired the emblem will need to complete the requirements after this patch goes live in order to equip the emblem.\n\n\nUser: What is the new change regarding Gyrfalcon's Hauberk's reserve overshield?\nAI: Gyrfalcon's Hauberk's reserve overshield now deploys when a player uses Ensnaring Slam. This is a change introduced in the latest patch.\n\n\nUser: What were the issues fixed regarding Arbor Warden?\nAI: Two issues related to Arbor Warden were fixed in this patch. One issue caused the 